In [1]:
import pandas as pd

# Define the URLs
url_female = "https://raw.githubusercontent.com/kocsigabor99/MAJOR-CROPS-FAODATA/main/UN_PPP2022_Forecast_PopulationBySingleAge_Female.csv"
url_male = "https://raw.githubusercontent.com/kocsigabor99/MAJOR-CROPS-FAODATA/main/UN_PPP2022_Forecast_PopulationBySingleAge_Male.csv"

# Read the CSV files into DataFrames
df_female = pd.read_csv(url_female)
df_male = pd.read_csv(url_male)

# Add a column to identify gender
df_female['Gender'] = 'Female'
df_male['Gender'] = 'Male'

# Merge the DataFrames
df_merged = pd.concat([df_female, df_male], ignore_index=True)

# Display the merged DataFrame
df_merged.head()

,"Region, subregion, country or area",Year,Gender,0,1,2,3,4,5,6,...,91,92,93,94,95,96,97,98,99,100+
0,WORLD,2022,Female,63727.730,63522.796,64218.110,65045.340,65918.313,66578.123,66380.278,...,3019.316,2469.187,1955.831,1513.472,1153.833,860.832,620.766,441.617,301.316,511.744
1,WORLD,2023,Female,63871.371,63325.641,63340.047,64085.254,64935.401,65823.183,66495.612,...,3084.121,2532.712,2033.361,1578.133,1195.901,892.319,649.811,456.299,316.094,547.105
2,WORLD,2024,Female,64058.216,63483.538,63144.314,63208.219,63976.258,64841.159,65741.583,...,3185.672,2604.360,2099.052,1651.472,1254.673,929.658,677.456,480.453,327.902,581.583
3,WORLD,2025,Female,64212.898,63677.215,63302.499,63012.362,63099.339,63882.338,64759.952,...,3291.202,2694.689,2162.846,1708.387,1315.784,977.363,706.863,502.097,346.271,613.880
4,WORLD,2026,Female,64361.668,63838.842,63496.874,63170.806,62903.842,63005.968,63801.776,...,3428.534,2787.235,2240.305,1762.898,1363.231,1026.542,744.252,524.413,362.579,649.381


In [2]:
# Load the birth rate data
birth_rate_url = 'https://raw.githubusercontent.com/kocsigabor99/MAJOR-CROPS-FAODATA/main/UN_PPP2022_Birth%20rate_Single_Year.csv'

# Load the birth rate data with low_memory=False to avoid DtypeWarning
birth_rate_df = pd.read_csv(birth_rate_url, low_memory=False)

# Convert all columns except 'Year' and 'Region, subregion, country or area' to numeric, handling errors
for col in birth_rate_df.columns[2:]:
    birth_rate_df[col] = pd.to_numeric(birth_rate_df[col], errors='coerce')

# Drop rows with missing or non-numeric values in the birth rate columns
birth_rate_df = birth_rate_df.dropna(subset=birth_rate_df.columns[2:])

# Merge birth rate data with population data based on 'Year' and 'Region, subregion, country or area'
merged_df = pd.merge(df_female, birth_rate_df, on=['Year', 'Region, subregion, country or area'], suffixes=('', '_y'), how='inner')

# Calculate the number of births for each age group
for age in range(15, 50):  # Assuming age ranges from 15 to 49
    age_str = str(age)
    birth_rate_col = f'{age_str}_y'
    if age_str in merged_df.columns and birth_rate_col in merged_df.columns:
        merged_df[f'Births_{age}'] = merged_df[age_str] * merged_df[birth_rate_col]

# Sum up the number of births across all age groups to get total births
birth_columns = [f'Births_{age}' for age in range(15, 50) if f'Births_{age}' in merged_df.columns]
merged_df['Total_Births'] = merged_df[birth_columns].sum(axis=1)

# Select columns to display in the final DataFrame
columns_to_display = ['Year', 'Region, subregion, country or area', 'Total_Births'] + birth_columns

# Display the DataFrame with total births and births per age group
print(merged_df[columns_to_display])

       Year Region, subregion, country or area  Total_Births      Births_15  \
0      2022                              WORLD  1.334320e+08  816746.581912   
1      2023                              WORLD  1.337286e+08  813653.314456   
2      2024                              WORLD  1.339417e+08  809100.166602   
3      2025                              WORLD  1.341781e+08  802883.870424   
4      2026                              WORLD  1.343820e+08  795990.589590   
...     ...                                ...           ...            ...   
22668  2096          Wallis and Futuna Islands  8.574207e+01       0.006486   
22669  2097          Wallis and Futuna Islands  8.562472e+01       0.006716   
22670  2098          Wallis and Futuna Islands  8.519468e+01       0.006644   
22671  2099          Wallis and Futuna Islands  8.490596e+01       0.006908   
22672  2100          Wallis and Futuna Islands  8.439489e+01       0.007172   

          Births_16     Births_17     Births_18    

In [3]:
# Calculate the number of pregnant women per single age group
pregnant_cols = {f'Pregnant_{age}': merged_df[f'Births_{age}'] for age in range(15, 50)}

# Extrapolate for pregnant teens aged 14 based on births in age 15
pregnant_cols['Pregnant_14'] = merged_df['Births_15']

# Calculate the number of breastfeeding women per single age group
breastfeeding_cols = {f'Breastfeeding_{age}': merged_df[f'Births_{age-2}'] + merged_df[f'Births_{age-1}'] for age in range(17, 50)}

# Special cases for ages 15 and 16
breastfeeding_cols['Breastfeeding_15'] = merged_df['Births_15']
breastfeeding_cols['Breastfeeding_16'] = merged_df['Births_15'] + merged_df['Births_16']

# Create a new DataFrame with these columns
new_cols = {**pregnant_cols, **breastfeeding_cols}
new_df = pd.DataFrame(new_cols)

# Concatenate the new DataFrame with the original DataFrame
merged_df = pd.concat([merged_df, new_df], axis=1)

# Optionally, save the updated DataFrame to a new CSV file
merged_df.to_csv('updated_birth_data.csv', index=False)

columns_list = merged_df.columns.tolist()
print(columns_list)

['Region, subregion, country or area', 'Year', 'Gender', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100+', '15_y', '16_y', '17_y', '18_y', '19_y', '20_y', '21_y', '22_y', '23_y', '24_y', '25_y', '26_y', '27_y', '28_y', '29_y', '30_y', '31_y', '32_y', '33_y', '34_y', '35_y', '36_y', '37_y', '38_y', '39_y', '40_y', '41_y', '42_y', '43_y', '44_y', '45_y', '46_y', '47_y', '48_y', '49_y', 'Births_15', 'Births_16', 'Births_17', 'Births_18', 'Births_19', 

In [4]:
# Set pandas display option to show all columns
pd.set_option('display.max_columns', None)

# Filter the DataFrame to show the row for Lesotho in 2022
lesotho_2022 = merged_df[(merged_df['Year'] == 2022) & (merged_df['Region, subregion, country or area'] == 'Lesotho')]

# Display the entire row for Lesotho in 2022
print(lesotho_2022)

     Region, subregion, country or area  Year  Gender       0       1       2  \
5214                            Lesotho  2022  Female  28.048  27.343  27.136   

           3      4       5       6       7       8       9      10      11  \
5214  27.021  26.92  26.844  26.603  26.333  26.279  26.003  25.496  24.785   

          12      13      14     15      16      17      18      19      20  \
5214  24.253  24.221  24.201  23.72  23.095  22.936  23.076  23.014  22.504   

          21      22      23      24      25      26      27      28      29  \
5214  21.919  21.584  21.558  21.655  21.504  20.991  20.466  20.194  20.132   

          30      31      32      33      34      35      36      37      38  \
5214  19.979  19.616  19.245  18.898  18.388  17.535  16.636  15.909  15.137   

          39     40      41      42      43      44      45     46     47  \
5214  14.314  13.51  12.757  12.055  11.396  10.807  10.223  9.585  8.962   

        48     49     50     51     52    

In [4]:
# 2. Subtract the numbers in the 'Pregnant_14' to 'Pregnant_49' columns from the corresponding female single age groups
pregnant_columns = {f'Pregnant_{age}': age for age in range(14, 50)}
for col, age in pregnant_columns.items():
    merged_df[str(age)] -= merged_df[col] / 1000

# 3. Subtract the numbers in the 'Breastfeeding_15' to 'Breastfeeding_49' columns from the corresponding female single age groups
breastfeeding_columns = {f'Breastfeeding_{age}': age for age in range(15, 50)}
for col, age in breastfeeding_columns.items():
    merged_df[str(age)] -= merged_df[col] / 1000

# Optionally, save the updated DataFrame to a new CSV file
merged_df.to_csv('updated_nutritional_data.csv', index=False)

# Print the columns of the updated DataFrame
columns_list = merged_df.columns.tolist()
print(columns_list)

['Region, subregion, country or area', 'Year', 'Gender', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100+', '15_y', '16_y', '17_y', '18_y', '19_y', '20_y', '21_y', '22_y', '23_y', '24_y', '25_y', '26_y', '27_y', '28_y', '29_y', '30_y', '31_y', '32_y', '33_y', '34_y', '35_y', '36_y', '37_y', '38_y', '39_y', '40_y', '41_y', '42_y', '43_y', '44_y', '45_y', '46_y', '47_y', '48_y', '49_y', 'Births_15', 'Births_16', 'Births_17', 'Births_18', 'Births_19', 

In [ ]:
# Set pandas display option to show all columns
pd.set_option('display.max_columns', None)

# Filter the DataFrame to show the row for Lesotho in 2022
lesotho_2022 = merged_df[(merged_df['Year'] == 2022) & (merged_df['Region, subregion, country or area'] == 'Lesotho')]

# Display the entire row for Lesotho in 2022
print(lesotho_2022)

In [5]:
rounded_df = merged_df.round(10)

In [6]:
# Create a copy of the DataFrame to apply scaling
scaled_df = rounded_df.copy()

# Define columns to scale (pregnant and breastfeeding columns)
columns_to_scale = [f'Pregnant_{age}' for age in range(14, 50)] + [f'Breastfeeding_{age}' for age in range(15, 50)]

# Scale the specified columns by dividing by 1000
scaled_df[columns_to_scale] /= 1000

In [ ]:
# Set pandas display option to show all columns
pd.set_option('display.max_columns', None)

# Filter the DataFrame to show the row for Lesotho in 2022
lesotho_2022 = scaled_df[(scaled_df['Year'] == 2022) & (scaled_df['Region, subregion, country or area'] == 'Lesotho')]

# Display the entire row for Lesotho in 2022
print(lesotho_2022)

In [7]:
import pandas as pd

# Example DataFrame (replace with your actual DataFrame)
# scaled_df = ...

# Columns to drop and save
columns_to_save = ['15_y', '16_y', '17_y', '18_y', '19_y', '20_y', '21_y', '22_y', '23_y', '24_y',
                   '25_y', '26_y', '27_y', '28_y', '29_y', '30_y', '31_y', '32_y', '33_y', '34_y',
                   '35_y', '36_y', '37_y', '38_y', '39_y', '40_y', '41_y', '42_y', '43_y', '44_y',
                   '45_y', '46_y', '47_y', '48_y', '49_y',
                   'Births_15', 'Births_16', 'Births_17', 'Births_18', 'Births_19', 'Births_20',
                   'Births_21', 'Births_22', 'Births_23', 'Births_24', 'Births_25', 'Births_26',
                   'Births_27', 'Births_28', 'Births_29', 'Births_30', 'Births_31', 'Births_32',
                   'Births_33', 'Births_34', 'Births_35', 'Births_36', 'Births_37', 'Births_38',
                   'Births_39', 'Births_40', 'Births_41', 'Births_42', 'Births_43', 'Births_44',
                   'Births_45', 'Births_46', 'Births_47', 'Births_48', 'Births_49']

# Create a new DataFrame to save the dropped columns
saved_data = scaled_df[columns_to_save].copy()

# Drop the columns from scaled_df
scaled_df.drop(columns_to_save, axis=1, inplace=True)

# Verify that scaled_df now does not contain the dropped columns
print(scaled_df.columns.tolist())

# Verify that saved_data contains the dropped columns for later use
print(saved_data.columns.tolist())

['Region, subregion, country or area', 'Year', 'Gender', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100+', 'Total_Births', 'Pregnant_15', 'Pregnant_16', 'Pregnant_17', 'Pregnant_18', 'Pregnant_19', 'Pregnant_20', 'Pregnant_21', 'Pregnant_22', 'Pregnant_23', 'Pregnant_24', 'Pregnant_25', 'Pregnant_26', 'Pregnant_27', 'Pregnant_28', 'Pregnant_29', 'Pregnant_30', 'Pregnant_31', 'Pregnant_32', 'Pregnant_33', 'Pregnant_34', 'Pregnant_35', 'Pregnant_36',

In [8]:
# Adjust the merge keys as per your actual column names
merge_keys = ['Region, subregion, country or area', 'Year',]

# Perform inner merge on the common keys
merged_data = pd.merge(scaled_df, df_male, on=merge_keys, suffixes=('_female', '_male'))

In [9]:
# Assuming merged_data is your DataFrame with the specified columns
merged_data = merged_data.drop(columns=['Gender_female', 'Gender_male'])

In [11]:
columns_list = merged_data.columns.tolist()
print(columns_list)

['Region, subregion, country or area', 'Year', '0_female', '1_female', '2_female', '3_female', '4_female', '5_female', '6_female', '7_female', '8_female', '9_female', '10_female', '11_female', '12_female', '13_female', '14_female', '15_female', '16_female', '17_female', '18_female', '19_female', '20_female', '21_female', '22_female', '23_female', '24_female', '25_female', '26_female', '27_female', '28_female', '29_female', '30_female', '31_female', '32_female', '33_female', '34_female', '35_female', '36_female', '37_female', '38_female', '39_female', '40_female', '41_female', '42_female', '43_female', '44_female', '45_female', '46_female', '47_female', '48_female', '49_female', '50_female', '51_female', '52_female', '53_female', '54_female', '55_female', '56_female', '57_female', '58_female', '59_female', '60_female', '61_female', '62_female', '63_female', '64_female', '65_female', '66_female', '67_female', '68_female', '69_female', '70_female', '71_female', '72_female', '73_female', '

In [10]:
# Load your data
url_nutritional = 'https://raw.githubusercontent.com/kocsigabor99/MAJOR-CROPS-FAODATA/main/Micronutrient%20Needs%20by%20single%20year%20Pregnant%20and%20Breastfeeding%20included.csv'
nutritional_df = pd.read_csv(url_nutritional)

columns_list2 = nutritional_df.columns.tolist()
print(columns_list2)

['Age', 'Gender', 'Breastfeeding/Pregnant', 'Vitamin A', 'Vitamin B1', 'Vitamin B2', 'Vitamin B3', 'Vitamin B5', 'Vitamin B6', 'Vitamin B7', 'Vitamin B9', 'Vitamin B12', 'Vitamin C', 'Vitamin D', 'Vitamin E', 'Vitamin K', 'Calcium', 'Copper', 'Iodine', 'Iron heme', 'Iron (non-heme)', 'Magnesium', 'Manganese', 'Molybdenum', 'Phosporus', 'Potassium', 'Selenium', 'Zinc']


In [11]:
import pandas as pd

# Load your data
url_nutritional = 'https://raw.githubusercontent.com/kocsigabor99/MAJOR-CROPS-FAODATA/main/Micronutrient%20Needs%20by%20single%20year%20Pregnant%20and%20Breastfeeding%20included.csv'
nutritional_df = pd.read_csv(url_nutritional)

# Initialize a dictionary to collect results
result_dict = {
    'Region, subregion, country or area': merged_data['Region, subregion, country or area'],
    'Year': merged_data['Year']
}

# Function to add nutrient data to the result_dict
def add_nutrient_data(age_group, gender, condition=None):
    nutrient_requirements = nutritional_df[
        (nutritional_df['Age'] == str(age_group)) & 
        (nutritional_df['Gender'] == gender)
    ]
    
    if condition:
        nutrient_requirements = nutrient_requirements[
            (nutrient_requirements['Breastfeeding/Pregnant'] == condition)
        ]
    else:
        nutrient_requirements = nutrient_requirements[
            (nutrient_requirements['Breastfeeding/Pregnant'].isnull()) | 
            (nutrient_requirements['Breastfeeding/Pregnant'] == 'No')
        ]
    
    # Debug print statements
    print(f"Age Group: {age_group}, Gender: {gender}, Condition: {condition}")
    print(f"Filtered Nutrient Requirements:\n{nutrient_requirements}\n")

    if not nutrient_requirements.empty:
        daily_requirements = nutrient_requirements.iloc[0]  # Assuming only one row matches the filter

        for nutrient in daily_requirements.index[3:]:  # Skip the first three columns (Age, Gender, Breastfeeding/Pregnant)
            if condition:
                column_name = f'{condition}_{age_group}'
            else:
                column_name = f'{age_group}_{gender.lower()}'
            
            result_dict[f'{column_name}_{nutrient}'] = merged_data[column_name] * daily_requirements[nutrient]

# Extract age groups from merged_data column headers
age_groups_female = sorted(set([col.split('_')[0] for col in merged_data.columns if col.endswith('_female')]))
age_groups_male = sorted(set([col.split('_')[0] for col in merged_data.columns if col.endswith('_male')]))

# Step 1: Calculate nutrient requirements for all age groups and genders
for age_group in age_groups_female:
    print(f"Processing Female Age Group: {age_group}")
    add_nutrient_data(age_group, 'Female')

    # Add conditions if applicable
    add_nutrient_data(age_group, 'Female', 'Breastfeeding')
    add_nutrient_data(age_group, 'Female', 'Pregnant')

for age_group in age_groups_male:
    print(f"Processing Male Age Group: {age_group}")
    add_nutrient_data(age_group, 'Male')

# Create the result_df DataFrame from result_dict
result_df = pd.DataFrame(result_dict)

Processing Female Age Group: 0
Age Group: 0, Gender: Female, Condition: None
Filtered Nutrient Requirements:
    Age  Gender Breastfeeding/Pregnant  Vitamin A  Vitamin B1  Vitamin B2  \
101   0  Female                     No        500         0.3         0.3   

     Vitamin B3  Vitamin B5  Vitamin B6  Vitamin B7  ...  Iodine  Iron heme  \
101           4         1.8         0.3           6  ...     130         11   

     Iron (non-heme)  Magnesium  Manganese  Molybdenum  Phosporus  Potassium  \
101               22         75        0.6           3        275        860   

     Selenium  Zinc  
101        20   220  

[1 rows x 28 columns]

Age Group: 0, Gender: Female, Condition: Breastfeeding
Filtered Nutrient Requirements:
Empty DataFrame
Columns: [Age, Gender, Breastfeeding/Pregnant, Vitamin A, Vitamin B1, Vitamin B2, Vitamin B3, Vitamin B5, Vitamin B6, Vitamin B7, Vitamin B9, Vitamin B12, Vitamin C, Vitamin D, Vitamin E, Vitamin K, Calcium, Copper, Iodine, Iron heme, Iron (non-

In [12]:
# Set pandas display option to show all columns
pd.set_option('display.max_columns', None)

# Filter the DataFrame to show the row for Lesotho in 2022
lesotho_2022 = result_df[(result_df['Year'] == 2022) & (result_df['Region, subregion, country or area'] == 'Lesotho')]

# Display the entire row for Lesotho in 2022
print(lesotho_2022)

     Region, subregion, country or area  Year  0_female_Vitamin A  \
5372                            Lesotho  2022             14024.0   

      0_female_Vitamin B1  0_female_Vitamin B2  0_female_Vitamin B3  \
5372               8.4144               8.4144              112.192   

      0_female_Vitamin B5  0_female_Vitamin B6  0_female_Vitamin B7  \
5372              50.4864               8.4144              168.288   

      0_female_Vitamin B9  0_female_Vitamin B12  0_female_Vitamin C  \
5372              2243.84                14.024              1402.4   

      0_female_Vitamin D  0_female_Vitamin E  0_female_Vitamin K  \
5372              280.48              140.24               70.12   

      0_female_Calcium  0_female_Copper  0_female_Iodine  0_female_Iron heme  \
5372           7292.48          6170.56          3646.24             308.528   

      0_female_Iron (non-heme)  0_female_Magnesium  0_female_Manganese  \
5372                   617.056              2103.6          